In [ ]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""

- 토큰화

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts([text])

t.word_docs

defaultdict(int,
            {'경마장에': 1,
             '오는': 1,
             '뛰고': 1,
             '가는': 1,
             '법이다': 1,
             '고와야': 1,
             '있다': 1,
             '그의': 1,
             '말이': 1,
             '곱다': 1,
             '있는': 1})

In [ ]:
# 단어의 수를 저장
vocab_size = len(t.word_docs) + 1

vocab_size

12

- 정수 인코딩

In [ ]:
seq = []
# 문장을 한 줄씩 읽어온다
for line in text.split("\n"):
  encoded = t.texts_to_sequences([line])[0]
  # 조합 : 가능한 단어 조합을 생성
  for i in range(1, len(encoded)) :
    s = encoded[:i+1]
    seq.append(s)
seq
# 경마장에 있는 말이 뛰고 있다
#    경마장에 있는
#    경마장에 있는 말이
#    경마장에 있는 말이 뛰고
#    경마장에 있는 말이 뛰고 있다
# 그의 말이 법이다
#    그의 말이
#    그의 말이 법이다
# 가는 말이 고와야 오는 말이 곱다
#    가는 말이
#    가는 말이 고와야
#    가는 말이 고와야 오는
#    가는 말이 고와야 오는 말이
#    가는 말이 고와야 오는 말이 곱다

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

- 인코딩된 결과를 같은 길이로 만듬

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 6

sequence = pad_sequences(seq, maxlen=max_len)
sequence

# 맨 끝의 숫자가 라벨데이터, 나머지 특성데이터

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

- 특성데이터와 라벨데이터로 분리

In [ ]:
X = sequence[:, :-1]
y = sequence[:, -1]

X.shape, y.shape

((11, 5), (11,))

In [ ]:
# 값이 수치 형태의 데이터인 경우에만 원핫인코딩
from tensorflow.keras.utils import to_categorical

# 항상 인코딩은 전체 단어 수만큼으로 인코딩 크기를 설정해주서야 한다.
y_en = to_categorical(y, num_classes=vocab_size)
y_en.shape

(11, 12)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
y_en

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

- 신경망 설계

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model1 = Sequential()

model1.add(Embedding(vocab_size, 12, input_length=max_len-1))

model1.add(SimpleRNN(32))

model1.add(Dense(12, activation="softmax"))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 12)             144       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1440      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=["accuracy"])

In [ ]:
h1 = model1.fit(X, y_en, epochs=200)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 2.5085 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 2.4937 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 2.4791 - accuracy: 0.0909
Epoch 4/200
1/1 [==============================] - 0s 14ms/step - loss: 2.4646 - accuracy: 0.0909
Epoch 5/200
1/1 [==============================] - 0s 15ms/step - loss: 2.4501 - accuracy: 0.0909
Epoch 6/200
1/1 [==============================] - 0s 13ms/step - loss: 2.4355 - accuracy: 0.1818
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 2.4207 - accuracy: 0.2727
Epoch 8/200
1/1 [==============================] - 0s 14ms/step - loss: 2.4057 - accuracy: 0.1818
Epoch 9/200
1/1 [==============================] - 0s 13ms/step - loss: 2.3904 - accuracy: 0.1818
Epoch 10/200
1/1 [==============================] - 0s 13ms/step - loss: 2.3746 - accuracy: 0.1818
Epoch 11/200


- 문장을 생성하는 함수

In [ ]:
import numpy as np
# model : 사용한 모델
# t : 사용한 Tonkenizer
# word : 입력단어
# n : 입력단어로부터 예측할 단어 수
def generate_seqence(model, t, current_word, n) :
  init_word = current_word # 입력된 단어로 시작해야 하므로 입력단어를 먼저 저장
  
  # 생성한 문장을 저장할 변수 선언
  seq=""

  for _ in range(n):
    # 현재 단어를 인코딩하고 padding 처리를 수행
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1)

    # 현재 단어로 다음 단어를 예측 
    result = np.argmax(model.predict(encoded))

    #print(result)
    # 예측한 단어와 동일한 인덱스의 단어가 있다면
    for word, index in t.word_index.items():
      if index == result :
        break
    # 현재단어와 예측한 단어를 연결
    current_word = current_word + " " + word

    seq = seq + " " + word

  seq = init_word + seq

  return seq

In [ ]:
generate_seqence(model1, t, "가는", 2)

1/1 [==============================] - 0s 18ms/step


'가는 말이 고와야'

# 실제 기사들을 이용하여 문장 생성하기
  - 빅카이즈 : https://www.bigkinds.or.kr/ 에서 "인공지능" 검색

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 562 kB/s 
     |████████████████████████████████| 453 kB 68.7 MB/s 


In [27]:
# 기사 읽기
import pandas as pd

file_dir = ""
news = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/사물지능_딥러닝_2022/data/NewsResult_20220727-20221027.xlsx")

news.head()

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100311e+06,20221027,서울경제,김동호 기자,이젠 국회에서도 인공지능 로봇 ‘큐아이’ 만난다,IT_과학>IT_과학일반,정치>행정_자치,NaN,NaN,NaN,NaN,"홍희경,문정원,이하,이광재",국립박물관,"국회사무처,큐아이,국회박물관,국회,한국문화정보원-국회사무처,대국","이젠,국회,큐아이,인공,지능,로봇,한국문화정보원,국회사무처,국회박물관,큐레이팅봇,서...","큐아이,박물관,국회사무처,관람객,국회박물관,대국민,한국문화정보원,업무협약,문정원,문...","한국문화정보원(원장 홍희경, 이하 문정원)과 국회사무처(총장 이광재)는 국회박물관에...",http://www.sedaily.com/NewsView/26CIGRZ2IM,NaN
1,8.100101e+06,20221027,KBS,주아랑,드론 인공위성 융합 기술로 저수지 가뭄 관리,지역>울산,지역>경남,지역>전북,NaN,NaN,NaN,NaN,"울산,경상권,첨단","울산,국립재난안전연구원,유럽우주국,국립재난안전연구원 재난정보연구관","드론,인공위성,융합,관리,저수지,가뭄,앵커,기후,변화,빈도,가뭄,발생,체계적,가뭄,...","저수지,드론,연구원,인공위성,국립재난안전연구원,울산,주아랑,이종설,경상,연구관,인공...",[KBS 울산]\n [앵커]\n\n 급격한 기후 변화로 가뭄 발생 빈도가 높아지면서...,https://news.kbs.co.kr/news/view.do?ncd=558790...,NaN
2,7.100501e+06,20221027,전자신문,김한식,"국립광주과학관, '가족과 함께하는 AI 과학골든벨' 성황리 개최",지역>전남,지역>광주,지역>대전,NaN,NaN,NaN,전태호,"광주시,성황리,결승전","광주시,브론즈벨,국립광주과학관(,국립광주과학관,북구","국립광주과학관,가족,AI,과학,골든벨,성황리,개최,국립광주과학관,관장직무대리전태호,...","인공지능,광주시,골든벨,국립광주과학관,광주과학관,관장직무대리전태호,전태호,과학골든벨...",국립광주과학관(관장직무대리전태호)은 22일 국립광주과학관 야외에서 '가족과 함께하는...,http://www.etnews.com/20221027000005,NaN
3,1.100201e+06,20221027,국민일보,웨더봇,오늘의 날씨 - 오전 (2022년 10월 27일),사회>날씨,지역>광주,NaN,NaN,NaN,NaN,NaN,"수원,강릉,전주,대전,서울,청주,부산,제주,대구,인천,춘천,광주",NaN,"날씨,27일,목요일,날씨,최저,서울,인천,수원,춘천,강릉,청주,대전,전주,광주,대구...","부산,제주,강릉,대구,춘천,광주,수원,전주,인천,서울,청주,목요일,최고기온,날씨,대...","10월 27일 목요일, 아침 날씨입니다. \n \n아침 최저기온은 서울 8.0도, ...",https://news.kmib.co.kr/article/view.asp?arcid...,예외
4,1.100611e+06,20221027,서울신문,NaN,"실리콘밸리에 불어닥친 한파 투자도, 혁신도, 삶의 질도 ‘겨울’[손재권의 실리콘밸리...",경제>국제경제,경제>취업_창업,경제>자동차,NaN,NaN,NaN,블룸버그,"실리콘밸리,우크라이나,최고경영자,미국,경기,아일랜드,핸콕,미,러시아","구글,실리콘밸리,다이내믹,미국,연준,넷플릭스,트위터,머스크,애플,마이크로소프트,유튜...","실리콘밸리,한파,투자,혁신,겨울,손재권,실리콘밸리,투데이,닷컴버블,2000년,변화,...","실리콘밸리,테슬라,구글,마이크로소프트,미국,강달러,메타,응답자,텐서,인플레이션,연준...",미국 실리콘밸리는 닷컴버블이 터진 지난 2000년 이후 가장 큰 변화와 충격을 겪고...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN


- 형태소 분리 및 불용어 처리 

In [28]:
from konlpy.tag import Okt

okt= Okt()

# 제목, 본문 컬럼

# 제목 컬럼에서 한글, 숫자, 빈공간 추출
news["제목"] = news["제목"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [29]:
headlines = []
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와', '등', '으로도']

# 제목을 하나씩 읽어와서 형태소 분리를 하고 불용어 제거
for sentence in news["제목"]:
  temp = []
  # 형태소 분리, 언간추출
  temp = okt.morphs(sentence, stem=True)

  #불용어 처리
  temp = [word for word in temp if not word in stopwords]
  headlines.append(temp)

In [30]:
headlines[:5]

[['젠', '국회', '에서도', '인공', '지능', '로봇', '큐', '아이', '만나다'],
 ['드론', '인공위성', '융합', '기술', '로', '저수지', '가뭄', '관리'],
 ['국립광주과학관', '가족', '함께', '하다', '과학', '골든벨', '성황리', '개최'],
 ['오늘', '날씨', '오전', '2022년', '10월', '27일'],
 ['실리콘밸리',
  '불어',
  '닥치다',
  '한파',
  '투자',
  '혁신',
  '삶',
  '질도',
  '겨울',
  '손재',
  '권',
  '실리콘밸리',
  '투데이']]

- 인코딩

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_features = 5000

t= Tokenizer(num_words = max_features)
# 빈도수 분석 -> 정렬 -> 순서대로 인덱스 부여 (1 부터)

t.fit_on_texts(headlines)

# 단어 수
vocab_size = len(t.word_index) + 1

# 인코딩
headlines_en = t.texts_to_sequences(headlines)

headlines_en[:5]

[[459, 460, 1634, 2, 1, 31, 1262, 222, 262],
 [182, 1952, 79, 4, 5, 1635, 1419, 177],
 [570, 931, 305, 3, 102, 1420, 1421, 11],
 [14, 20, 66, 15, 108, 712],
 [484, 2397, 3054, 1636, 26, 22, 660, 4351, 3055, 4352, 274, 484, 1637]]

- 2개 이상의 단어 조합으로 

In [34]:
seqs=[]

for line in headlines_en :
  for i in range(1, len(line)):
    seq = line[:i+1]
    seqs.append(seq)
seqs[:10]

[[459, 460],
 [459, 460, 1634],
 [459, 460, 1634, 2],
 [459, 460, 1634, 2, 1],
 [459, 460, 1634, 2, 1, 31],
 [459, 460, 1634, 2, 1, 31, 1262],
 [459, 460, 1634, 2, 1, 31, 1262, 222],
 [459, 460, 1634, 2, 1, 31, 1262, 222, 262],
 [182, 1952],
 [182, 1952, 79]]

- 문장생성을 위해 인덱스별 단어를 저장

In [37]:
index_to_word = {}

for word, key in t.word_index.items():
  index_to_word[key] = word

index_to_word[30]

'인재'

In [38]:
# 문장의 최대 길이
max_len = max(len(s) for s in seqs)

max_len

33

- 같은 길이로 만들어줌

In [39]:
from numpy.lib.arraypad import pad
from tensorflow.keras.preprocessing.sequence import pad_sequences

seqs= pad_sequences(seqs, maxlen=max_len)

seqs[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  459,  460],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,  459,  460, 1634],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  459,  460, 1634,    2],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  459,  460, 1634,    2,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   

- 특성과 라벨데이터로 분리

In [40]:
X = seqs[:, :-1]
y = seqs[:, -1]

X.shape, y.shape

((42420, 32), (42420,))

In [41]:
from tensorflow.keras.utils import to_categorical

#  원핫인코딩
y_en = to_categorical(y, num_classes=vocab_size)

y_en.shape

(42420, 7989)

- 신경망 설계

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model2 = Sequential()

model2.add(Embedding(vocab_size, 20, input_length=max_len-1))

model2.add(LSTM(128))

model2.add(Dense(vocab_size, activation="softmax"))

model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 20)            159780    
                                                                 
 lstm (LSTM)                 (None, 128)               76288     
                                                                 
 dense (Dense)               (None, 7989)              1030581   
                                                                 
Total params: 1,266,649
Trainable params: 1,266,649
Non-trainable params: 0
_________________________________________________________________


In [44]:
model2.compile(loss="categorical_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])


In [57]:
h2=model2.fit(X,y_en, epochs=5)        

Epoch 1/5
1326/1326 [==============================] - 11s 8ms/step - loss: 0.6012 - accuracy: 0.8586
Epoch 2/5
1326/1326 [==============================] - 13s 10ms/step - loss: 0.5951 - accuracy: 0.8586
Epoch 3/5
1326/1326 [==============================] - 11s 8ms/step - loss: 0.5751 - accuracy: 0.8625
Epoch 4/5
1326/1326 [==============================] - 13s 10ms/step - loss: 0.5752 - accuracy: 0.8616
Epoch 5/5
1326/1326 [==============================] - 13s 9ms/step - loss: 0.5684 - accuracy: 0.8620


In [58]:
model2.evaluate(X,y_en)

1326/1326 [==============================] - 7s 5ms/step - loss: 0.4740 - accuracy: 0.8857


[0.4740310311317444, 0.8856671452522278]

In [59]:
import numpy as np
# model : 사용한 모델
# t : 사용한 Tonkenizer
# word : 입력단어
# n : 입력단어로부터 예측할 단어 수
def generate_seqence(model, t, current_word, n) :
  init_word = current_word # 입력된 단어로 시작해야 하므로 입력단어를 먼저 저장
  
  # 생성한 문장을 저장할 변수 선언
  seq=""

  for _ in range(n):
    # 현재 단어를 인코딩하고 padding 처리를 수행
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1)

    # 현재 단어로 다음 단어를 예측 
    result = np.argmax(model.predict(encoded))

    #print(result)
    # 예측한 단어와 동일한 인덱스의 단어가 있다면
    for word, index in t.word_index.items():
      if index == result :
        break
    # 현재단어와 예측한 단어를 연결
    current_word = current_word + " " + word

    seq = seq + " " + word

  seq = init_word + seq

  return seq

In [66]:
generate_seqence(model2, t, "체결", 20)

1/1 [==============================] - 0s 17ms/step


'체결 로봇 시대 기업 대응 초 격차 개발 로 장비 내 지 운영 지 기 되다 서울 교육 활성화 로 성공'